In [181]:
import os
import math
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [15]:
data_path = 'data/'

### Dataset Movie Lens

Load dataset from source

In [16]:
users = pd.read_csv(
    data_path + "ml-1m/users.dat",
    sep="::",
    names=["user_id", "sex", "age_group", "occupation", "zip_code"],
    engine="python",
)

ratings = pd.read_csv(
    data_path + "ml-1m/ratings.dat",
    sep="::",
    names=["user_id", "movie_id", "rating", "unix_timestamp"],
    engine="python",
)

movies = pd.read_csv(
    data_path + "ml-1m/movies.dat",
    sep="::",
    names=["movie_id", "title", "genres"],
    engine="python",
    encoding="latin-1",
)

In [17]:
mv = movies.copy()
us = users.copy()
rt = ratings.copy()

In [38]:
movies = mv.copy()
users = us.copy()
ratings = rt.copy()


## Preprocess

users:
* change la valeur str de la colonne sex en 1 ou 0 pour que le modèle comprenne la colonne
* drop la colonne zip_code car peu intéressante pour la tâche

movies:
* sectionne la partie année des titres pour créer une nouvelle colonne année
* supprime la partie (année) des titres pour qu'ils soient possible de les comparer avec le dataset imdb

ratings:
* passe la colonne rating de str a float
* drop la colonne "unix_timestamp" car inutile

In [39]:
# users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
# users["age_group"] = users["age_group"].apply(lambda x: f"group_{x}")
# users["occupation"] = users["occupation"].apply(lambda x: f"occupation_{x}")

users['sex'] = users['sex'].apply(lambda x: 1 if(x == 'M') else 0)
users.drop(columns="zip_code", inplace=True)

# movies["movie_id"] = movies["movie_id"].apply(lambda x: f"movie_{x}")
movies["year"] = movies["title"].apply(lambda x : int(x[-5:-1]))
movies["title"] = movies["title"].apply(lambda x : x[:-7].lower())

# ratings["movie_id"] = ratings["movie_id"].apply(lambda x: f"movie_{x}")
# ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

ratings.drop(columns="unix_timestamp", inplace=True)


Certains titres sont sous la forme: "aaaa, the" au lieu de "the aaaa". Je modifie donc la colonne title avec une regex, du pattern matching avec groupe, pour résoudre ce problème.
Après résolution du problème les titres pourront être comparé avec ceux de imdb.

In [43]:
pattern = re.compile(r'^(.*), (the|a|an)$', re.IGNORECASE)
def rearrange_title(title):
    match = pattern.match(title)
    if match:
        return f"{match.group(2)} {match.group(1)}"
    return title

movies['title'] = movies['title'].apply(rearrange_title)

Change le genre Children's avec Family pour correspondre au même genre que l'autre jeu de donnée (imdb)

In [42]:
movies['genres'] = movies['genres'].str.replace("Children's", "Family")

In [46]:
### pas besoin y en a pas
movies.drop_duplicates(inplace=True)

### dataset imdb

Data Loading

In [25]:
# name_basics = pd.read_csv(
#     data_path + r"imdb/name.basics.tsv",
#     sep='\t',
#     engine="python",
#     # low_memory=False
# )

title_basics = pd.read_csv(
    data_path + r"imdb/title.basics.tsv",
    sep='\t',
    # engine="python",
    low_memory=False
)

title_ratings = pd.read_csv(
    data_path + r"imdb/title.ratings.tsv",
    sep='\t',
    # engine="python",
    low_memory=False
)


je sélectionne que les critiques de films ou de tvmovie car movielens ne contient que ce genre d'oeuvre

In [142]:
movietype = ["movie", "tvMovie"]
title_preprocess = title_basics[title_basics["titleType"].isin(movietype)].copy()
title_preprocess['titleType'].unique()

array(['movie', 'tvMovie'], dtype=object)

In [143]:
### on drope endYear car la colonne ne fait sens que pour les TV series
title_preprocess.drop(columns="endYear", inplace=True)

Toutes les colonnes sont de type object, to_numeric c'est quand des valeurs sont a Nan

In [148]:
title_preprocess["primaryTitle"] = title_preprocess["primaryTitle"].apply(str.lower)
title_preprocess["originalTitle"] = title_preprocess["originalTitle"].apply(str.lower)
title_preprocess["isAdult"] = title_preprocess["isAdult"].apply(int)
title_preprocess['startYear'] = pd.to_numeric(title_preprocess['startYear'], errors="coerce")
title_preprocess['runtimeMinutes'] = pd.to_numeric(title_preprocess['runtimeMinutes'], errors="coerce")

title_ratings["averageRating"] = title_ratings['averageRating'].apply(float)
title_ratings["numVotes"] = title_ratings['numVotes'].apply(int)

title_preprocess.replace(r'\N', np.nan, inplace=True)
# title_preprocess.drop(columns='tconst', inplace=True)

merge les descriptions des films avec leur rating celon tconst, leur id

In [151]:
title_merged = pd.merge(title_preprocess, title_ratings, on="tconst", how="left")
title_merged.drop(columns="tconst", inplace=True)
title_merged.head()

,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,movie,miss jerry,miss jerry,0,1894.0,45.0,Romance,5.4,212.0
1,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0,1897.0,100.0,"Documentary,News,Sport",5.2,518.0
2,movie,bohemios,bohemios,0,1905.0,100.0,NaN,4.4,17.0
3,movie,the story of the kelly gang,the story of the kelly gang,0,1906.0,70.0,"Action,Adventure,Biography",6.0,914.0
4,movie,the prodigal son,l'enfant prodigue,0,1907.0,90.0,Drama,5.4,27.0


on merge les caractéristiques des films de imdb et movielens, selont les titres et l'année de parutions.

In [194]:
merged_df1 = pd.merge(movies, title_merged, left_on=["title", "year"], right_on=["primaryTitle", "startYear"], how="left")
merged_df2 = pd.merge(movies, title_merged, left_on=["title", "year"], right_on=["originalTitle", "startYear"], how="inner")
final_df = pd.concat([merged_df2, merged_df1], ignore_index=True).drop_duplicates()
# merged_df3 = pd.merge(movies, final_df, on='movie_id', how="outer")
# final_df = pd.concat([final_df, merged_df3], ignore_index=True).drop_duplicates()

j'ai deux cas soit title != primarytitle et j'obtiens un doublon a cause de left (je suis obligé sinon je perd des films), il y a deux films avec le meme titre et je prend le plus connue => qui est le plus plosible d'etre dans movielens.

In [197]:
def custom_rule(group):
    return group.sort_values(by="numVotes", ascending=False).iloc[0]

final_df = final_df.groupby("movie_id").apply(custom_rule).reset_index(drop=True)

merge les colonnes de genres de imdb et movie lens

In [199]:
def merge_genres(row):
    genres1 = row['genres_x'].split('|') if pd.notna(row['genres_x']) else []
    genres2 = row['genres_y'].split(',') if pd.notna(row['genres_y']) else []
    merged_genres = list(set(genres1 + genres2))  # Fusionner et supprimer les doublons
    return '|'.join(merged_genres)  # Rejoindre avec le délimiteur '|'

# Appliquer la fonction à chaque ligne du DataFrame
final_df['genres'] = final_df.apply(merge_genres, axis=1)

drop toutes les colonnes inutiles pour le model

In [201]:
final_df.drop(columns=["primaryTitle", "originalTitle", "genres_x", "genres_y", "startYear"], inplace=True)

ceux qui n'avaient pas de type dans imdb sont forcément des films vu qu'ils sont dans movie lens

In [202]:
final_df["titleType"] = final_df["titleType"].fillna("movie")

In [6]:
genres = set("|".join(final_df['genres'].unique()).split('|'))
for genre in genres:
    final_df[genre] = final_df['genres'].apply(lambda x: 1 if genre in x.split('|') else 0)
final_df["titleType"] = final_df["titleType"].apply(lambda x: 1 if x == "movie" else 0)

On remplit les averageRating avec ceux qu'on a dans MovieLens

In [26]:
for id in final_df[final_df["averageRating"].isna()]["movie_id"].unique():
    r_list = ratings[ratings["movie_id"] == id]
    if len(r_list) == 0:
        final_df.loc[final_df["movie_id"] == id, ["numVotes"]] = [0]
    else:
        final_df.loc[final_df["movie_id"] == id, ["averageRating","numVotes"]] = [r_list["rating"].values.mean(), len(r_list)]

In [206]:
final_df.to_csv("dataset.csv", index=False)

In [4]:
df = pd.read_csv("dataset.csv")

In [5]:
# df.drop(columns="startYear", inplace=True)
ratings

In [112]:
Y = ratings.pivot(index='movie_id', columns='user_id', values='rating')
Y = Y.fillna(0)
Y

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
genres = set("|".join(df['genres'].unique()).split('|'))
for genre in genres:
    users[genre] = 0
# Ajouter la colonne 'nrating'
users['nrating'] = 0


Crée un dataframe users avec le rating moyen pour chaque genre de film

In [107]:
# Fusionner les évaluations avec les genres
all_genres = set("|".join(df['genres'].unique()).split('|'))
merged_df = ratings.merge(df, on='movie_id', how="left")
users_df = users.copy().reset_index()

# Initialiser un dictionnaire pour stocker les sommes et les comptes des genres
genre_sum_count = {genre: {'sum': {}, 'count': {}} for genre in all_genres}

# Parcourir les lignes de merged_df pour calculer les sommes et les comptes
for _, row in merged_df.iterrows():
    user_id = row['user_id']
    rating = row['rating']
    genres = row['genres'].split('|')
    
    users_df.loc[users_df['user_id'] == user_id, 'nrating'] += 1
    
    for genre in genres:
        if user_id in genre_sum_count[genre]['sum']:
            genre_sum_count[genre]['sum'][user_id] += rating
            genre_sum_count[genre]['count'][user_id] += 1
        else:
            genre_sum_count[genre]['sum'][user_id] = rating
            genre_sum_count[genre]['count'][user_id] = 1

# Calculer les moyennes pour chaque genre et mettre à jour users_df
for genre in all_genres:
    for user_id in genre_sum_count[genre]['sum']:
        users_df.loc[users_df['user_id'] == user_id, genre] = (
            genre_sum_count[genre]['sum'][user_id] / genre_sum_count[genre]['count'][user_id]
        )

users_df.head()


      user_id  sex  age_group  occupation  Thriller   Fantasy   Musical  \
0           1    0          1          10  3.666667  4.100000  4.285714   
1           2    1         56          16  3.511628  3.625000  0.000000   
2           3    1         25          15  3.714286  3.428571  4.000000   
3           4    1         45           7  4.000000  3.500000  0.000000   
4           5    1         25          20  2.782609  3.266667  3.333333   
...       ...  ...        ...         ...       ...       ...       ...   
6035     6036    0         25          15  3.196721  3.135135  3.687500   
6036     6037    0         45           1  3.675676  3.800000  4.000000   
6037     6038    0         56           1  2.000000  4.000000  0.000000   
6038     6039    0         45           0  4.133333  3.733333  3.690476   
6039     6040    1         25           6  3.636364  2.807692  4.000000   

       History    Sci-Fi   Mystery  ...     Drama    Horror     Sport  \
0     5.000000  4.333333  

split le dataset en train et test

In [195]:
movies = df.drop(columns=["title", "genres"]).copy()

user_train, user_test = train_test_split(users_df, test_size=0.2, random_state=42)

movie_train, movie_test = train_test_split(movies, test_size=0.2, random_state=42)

In [225]:
y_train = ratings.pivot(index='movie_id', columns='user_id', values='rating').fillna(0)


In [231]:
scaler = StandardScaler()

user_train = transforme(users_df, "user_id", scaler)

movie_train = transforme(movies, "movie_id", scaler)

In [217]:
movies_c = movies[movies['movie_id'].isin(y_train.index)].copy()

In [196]:
user_id_train = np.array(user_train["user_id"])
movie_id_train = np.array(movie_train["movie_id"])

user_id_test = np.array(user_test["user_id"])
movie_id_test = np.array(movie_test["movie_id"])

ratings_train = ratings[ratings['user_id'].isin(user_id_train)].copy()
ratings_test = ratings[ratings['user_id'].isin(user_id_test)].copy()

ratings_train = ratings_train[ratings_train['movie_id'].isin(movie_id_train)]
ratings_test = ratings_test[ratings_test['movie_id'].isin(movie_id_test)]

# y_train = Y.drop(columns=[user_id_train])

In [197]:
y_train = ratings_train.pivot(index='movie_id', columns='user_id', values='rating')
y_train = y_train.fillna(0)

y_test = ratings_test.pivot(index='movie_id', columns='user_id', values='rating')
y_test = y_test.fillna(0)


In [198]:
def transforme(df, column, trans):
    df.set_index(column, inplace=True)
    return trans.fit_transform(df)

def getR(Y):
    R = Y.copy()
    R[R != 0] = 1
    return R

In [199]:
scaler = StandardScaler()

user_train = transforme(user_train, "user_id", scaler)
user_test = transforme(user_test, "user_id", scaler)

movie_train = transforme(movie_train, "movie_id", scaler)
movie_test = transforme(movie_test, "movie_id", scaler)

In [202]:
scalerY = MinMaxScaler((-1,1))

R_train = getR(y_train)
R_test = getR(y_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = scalerY.fit_transform(y_train.reshape(-1, 1))
y_test = scalerY.fit_transform(y_test.reshape(-1, 1))

In [203]:
num_user_features = user_train.shape[1]
num_item_features = movie_train.shape[1]

In [204]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential(
    [
        ### START CODE HERE ###
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(num_outputs),
        ### END CODE HERE ###
    ]
)

item_NN = tf.keras.models.Sequential(
    [
        ### START CODE HERE ###
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(num_outputs),
        ### END CODE HERE ###
    ]
)

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 26)]         0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 28)]         0           []                               
                                                                                                  
 sequential_10 (Sequential)     (None, 32)           43936       ['input_9[0][0]']                
                                                                                                  
 sequential_11 (Sequential)     (None, 32)           44448       ['input_10[0][0]']               
                                                                                            

In [205]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss=cost_fn)

In [206]:
user_train.shape

(4832, 26)

In [207]:
tf.random.set_seed(1)
model.fit(
    [user_train, movie_train],
    y_train,
    epochs=2,
    validation_data=([user_test, movie_test], y_test),
)

ValueError: Data cardinality is ambiguous:
  x sizes: 4832, 3106
  y sizes: 14254400
Make sure all arrays contain the same number of samples.